In [27]:
import pandas as pd
import json
import re

In [28]:
run_name = "faa_registry"
run_name = "cictt_registry"
run_name = "union"

# load JSONL
records = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_all.tsv") as f:
    for line in f:
        records.append(json.loads(line))


        # load JSONL
records_test = []
with open("../aircraft_er_predictions/" + run_name + "_predictions_test.tsv") as f:
    for line in f:
        records_test.append(json.loads(line))

In [29]:
df = pd.DataFrame(records)
df.head()

,left,right,match,match_confidence
0,COL make VAL FAIRCHILD COL model VAL 24 COL se...,COL make VAL FAIRCHILD COL model VAL 24 C8B,0,0.992935
1,COL make VAL CESSNA COL model VAL AT17 COL ser...,COL make VAL MNCOUP COL model VAL 110 COL seri...,0,0.999822
2,COL make VAL CESSNA COL model VAL 337 COL seri...,COL make VAL LR COL model VAL 36 COL series VA...,0,0.999902
3,COL make VAL MORANE SAULNIER COL model VAL MS8...,COL make VAL H COL model VAL 19 COL series VAL...,0,0.999853
4,COL make VAL MOONEY COL model VAL M18 COL seri...,COL make VAL CE COL model VAL 210 COL series V...,0,0.999400


In [30]:


gold = pd.read_csv("../data/ditto_aircraft/" + run_name + "/all_pairs.txt", sep="\t", header=None, names=["left", "right", "gold"])
print(gold.head())

                                                left  \
0  COL make VAL FAIRCHILD COL model VAL 24 COL se...   
1  COL make VAL CESSNA COL model VAL AT17 COL ser...   
2  COL make VAL CESSNA COL model VAL 337 COL seri...   
3  COL make VAL MORANE SAULNIER COL model VAL MS8...   
4  COL make VAL MOONEY COL model VAL M18 COL seri...   

                                               right  gold  
0        COL make VAL FAIRCHILD COL model VAL 24 C8B     0  
1  COL make VAL MNCOUP COL model VAL 110 COL seri...     0  
2  COL make VAL LR COL model VAL 36 COL series VA...     0  
3  COL make VAL H COL model VAL 19 COL series VAL...     0  
4  COL make VAL CE COL model VAL 210 COL series V...     0  


In [31]:
df["gold"] = gold["gold"]

In [32]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_true = df["gold"]
y_pred = df["match"]

print("Accuracy:", accuracy_score(y_true, y_pred))
print("\nClassification report:\n", classification_report(y_true, y_pred))
print("\nConfusion matrix:\n", confusion_matrix(y_true, y_pred))

Accuracy: 0.9893596807904237

Classification report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     30923
           1       0.96      0.97      0.97      5918

    accuracy                           0.99     36841
   macro avg       0.98      0.98      0.98     36841
weighted avg       0.99      0.99      0.99     36841


Confusion matrix:
 [[30701   222]
 [  170  5748]]


In [33]:
errors = df[df["gold"] != df["match"]]
print(errors[["left","right","gold","match","match_confidence"]].head(20))

                                                   left  \
13    COL make VAL BELL COL model VAL 205 COL series...   
211   COL make VAL BOEING COL model VAL 737 COL seri...   
947   COL make VAL EUROCOPTER COL model VAL BO105 CO...   
965   COL make VAL BOEING COL model VAL 727 COL seri...   
1092  COL make VAL DOUGLAS COL model VAL C54 COL ser...   
1202  COL make VAL BELL COL model VAL OH13 COL serie...   
1267  COL make VAL AIRBUS COL model VAL A321 COL ser...   
1529  COL make VAL AIRBUS HELICOPTERS COL model VAL ...   
1630  COL make VAL CESSNA COL model VAL 560 COL seri...   
1681  COL make VAL BELL COL model VAL 206 COL series...   
1702  COL make VAL BEECH COL model VAL RC45 COL seri...   
1734  COL make VAL ROCKWELL COL model VAL 100 COL se...   
1952  COL make VAL BOEING COL model VAL 727 COL seri...   
1965  COL make VAL ENSTROM COL model VAL F28 COL ser...   
2044  COL make VAL PIPER COL model VAL PA18 COL seri...   
2292  COL make VAL SIKORSKY COL model VAL S92 COL se... 

In [34]:
errors

,left,right,match,match_confidence,gold
13,COL make VAL BELL COL model VAL 205 COL series...,COL make VAL BELL COL model VAL 205A,1,0.718608,0
211,COL make VAL BOEING COL model VAL 737 COL seri...,COL make VAL B COL model VAL 737 COL series VA...,1,0.580589,0
947,COL make VAL EUROCOPTER COL model VAL BO105 CO...,COL make VAL MESSERSCHMITT-BOELKOW-BLOHM COL m...,0,0.708745,1
965,COL make VAL BOEING COL model VAL 727 COL seri...,COL make VAL BOEING COL model VAL 727-29C,0,0.966472,1
1092,COL make VAL DOUGLAS COL model VAL C54 COL ser...,COL make VAL DOUGLAS COL model VAL C54A-DC,0,0.570545,1
...,...,...,...,...,...
36688,COL make VAL ENSTROM COL model VAL 280 COL ser...,COL make VAL ENSTROM COL model VAL 280FX,0,0.788331,1
36717,COL make VAL CESSNA COL model VAL 411 COL seri...,COL make VAL CE COL model VAL 402 COL series V...,1,0.837535,0
36753,COL make VAL KAWASAKI COL model VAL KV107 COL ...,COL make VAL KAWASAKI VERTOL COL model VAL KV1...,0,0.969157,1
36805,COL make VAL BELL COL model VAL UH1 COL series...,COL make VAL S M & T AIRCRAFT COL model VAL UH-1B,0,0.910225,1


In [35]:
errors.to_csv("../aircraft_er_predictions/" + run_name + "_errors_review.csv", index=False)

In [36]:
def parse_record(record: str):
    """Parse Ditto serialized record into a dict of {field: value}."""
    parts = re.split(r"COL |VAL ", record.strip())
    parts = [p for p in parts if p]  # drop empties
    return {parts[i].strip(): parts[i+1].strip() for i in range(0, len(parts), 2)}

parsed = []

for _, row in errors.iterrows():
    left = parse_record(row["left"])
    right = parse_record(row["right"])
    parsed.append({
        "cictt_make": left.get("make"),
        "make": right.get("make"),
        "cictt_model": left.get("model"),
        "model": right.get("model"),
        "cictt_series": left.get("series"),
        "series": right.get("series"),
        "cictt_cert": left.get("cert"),
        "cert": right.get("cert"),
        "cictt_name": left.get("name"),
        "name": right.get("name"),
        "gold": row["gold"],
        "predicted": row["match"],
        "confidence": row["match_confidence"]
    })

aligned = pd.DataFrame(parsed)

In [37]:
aligned.to_csv("../aircraft_er_predictions/" + run_name + "_aligned_errors_review.csv", index=False)